In [1]:
# %pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install numpy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [370]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [371]:
df = pd.read_csv('mental-heath-in-tech-2016_20161114.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

In [372]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 63 columns):
 #   Column                                                                                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                                                                                            --------------  -----  
 0   Are you self-employed?                                                                                                                                                            1433 non-null   int64  
 1   How many employees does your company or organization have?                                                                                                                        1146 non-null   object 
 2   Is your employer primarily a tech company/organization?     

In [373]:
# Checking for duplicated columns
duplicate_features = df.columns[df.T.duplicated()].tolist()

print("Duplicated features:")
print(duplicate_features)

Duplicated features:
[]


###### turns out there are no duplicated features, so we can move on

### Standardizing all missing values by replacing with NaN

In [374]:
import numpy as np

# Replace all common missing value representations with np.nan
df.replace(["NAN", "nan", "NA", "None", "?"], np.nan, inplace=True)
df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,6-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,6-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,6-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [375]:
# Check for missing values after replacement
print(df.isnull().sum())

Are you self-employed?                                                                  0
How many employees does your company or organization have?                            287
Is your employer primarily a tech company/organization?                               287
Is your primary role within your company related to tech/IT?                         1170
Does your employer provide mental health benefits as part of healthcare coverage?     287
                                                                                     ... 
What US state or territory do you live in?                                            593
What country do you work in?                                                            0
What US state or territory do you work in?                                            582
Which of the following best describes your work position?                               0
Do you work remotely?                                                                   0
Length: 63

In [376]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 63 columns):
 #   Column                                                                                                                                                                            Non-Null Count  Dtype  
---  ------                                                                                                                                                                            --------------  -----  
 0   Are you self-employed?                                                                                                                                                            1433 non-null   int64  
 1   How many employees does your company or organization have?                                                                                                                        1146 non-null   object 
 2   Is your employer primarily a tech company/organization?     

In [377]:
df.dtypes

Are you self-employed?                                                                 int64
How many employees does your company or organization have?                            object
Is your employer primarily a tech company/organization?                              float64
Is your primary role within your company related to tech/IT?                         float64
Does your employer provide mental health benefits as part of healthcare coverage?     object
                                                                                      ...   
What US state or territory do you live in?                                            object
What country do you work in?                                                          object
What US state or territory do you work in?                                            object
Which of the following best describes your work position?                             object
Do you work remotely?                                                 

### Standardazing Feature Values

In [378]:
import pandas as pd

# Load the dataset
df = pd.read_csv('mental-heath-in-tech-2016_20161114.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Normalize the data by converting to lowercase and stripping whitespace
df['gender_cleaned'] = df['What is your gender?'].str.lower().str.strip()

# Apply the gender mapping to clean and standardize the 'gender' column
gender_mapping = {
    'male': 'Male', 'm': 'Male', 'male.': 'Male', 'male (cis)': 'Male', 'cis male': 'Male',
    'cis-male': 'Male', 'cis man': 'Male', 'man': 'Male', 'male ': 'Male', 'm|': 'Male', 'malr': 'Male',
    'female': 'Female', 'f': 'Female', 'female ': 'Female', 'cis female': 'Female', 'cis-woman': 'Female',
    'woman': 'Female', 'female/woman': 'Female', 'i identify as female.': 'Female', 
    'fm': 'Female', 'female-bodied; no feelings about gender': 'Female', 
    'transgender woman': 'Transgender Female', 'transgender female': 'Transgender Female',
    'transitioned, m2f': 'Transgender Female', 'mtf': 'Transgender Female', 
    'male (trans, ftm)': 'Transgender Male', 'transgender male': 'Transgender Male', 
    'genderfluid (born female)': 'Non-binary', 'nonbinary': 'Non-binary', 'non-binary': 'Non-binary',
    'enby': 'Non-binary', 'bigender': 'Non-binary', 'genderflux': 'Non-binary', 'genderfluid': 'Non-binary', 
    'fluid': 'Non-binary', 'agender': 'Non-binary', 'genderqueer': 'Non-binary', 
    'genderqueer woman': 'Non-binary', 'androgynous': 'Non-binary', 'genderflux demi-girl': 'Non-binary',
    'afab': 'Non-binary', 'nb masculine': 'Non-binary', 'other/transfeminine': 'Other', 
    'male/genderqueer': 'Non-binary', 
    'queer': 'Non-binary', 
    'human': 'Other', 'unicorn': 'Other', 'none of your business': 'Prefer not to say', 
    'dude': 'Other', 'sex is male': 'Male', 'mail': 'Male', 'fem': 'Female', 
    'male 9:1 female, roughly': 'Other', 
    "i'm a man why didn't you make this a drop down question. you should of asked sex? and i would of answered yes please. seriously how much text can this take?": 'Other',
    'Other': 'Other', 'other': 'Other', 'prefer not to say': 'Prefer not to say',
    'female assigned at birth': 'Female',  # Standardizing to 'Female'
    'female or multi-gender femme': 'Non-binary',  # Standardizing to 'Non-binary'
    'cisgender female': 'Female',  # Standardizing to 'Female'
    'female (props for making this a freeform field, though)': 'Female',  # Standardizing to 'Female'
    'cisdude': 'Male'  # Standardizing to 'Male'
}

# Apply the mapping
df['gender_cleaned'] = df['gender_cleaned'].replace(gender_mapping)

# Ensure that 'nan' values are handled (could be replaced with 'Prefer not to say')
df['gender_cleaned'].fillna('Prefer not to say', inplace=True)

# Step 4: Overwrite the original 'What is your gender?' column with the cleaned 'gender_cleaned'
df['What is your gender?'] = df['gender_cleaned']

# Step 5: Drop the temporary 'gender_cleaned' column
df.drop(columns=['gender_cleaned'], inplace=True)

# Step 6: Save the updated DataFrame to the original CSV file (or a new one to preserve the original)
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Optionally, check if the changes are applied by printing unique values in 'What is your gender?'
print(df['What is your gender?'].unique())

['Male' 'Female' 'Non-binary' 'Transgender Female' 'Other'
 'Prefer not to say' 'Transgender Male']


In [379]:
import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Replace unrealistic ages (e.g., 99) with NaN
df['What is your age?'] = df['What is your age?'].apply(lambda x: np.nan if x == 99 else x)

# Step 2: Optionally, replace any other unrealistic ages (e.g., <18 or >100)
df['What is your age?'] = df['What is your age?'].apply(lambda x: np.nan if x < 18 or x > 100 else x)

# Save the cleaned dataset
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Check the modified values
print(df['What is your age?'].unique())

[39. 29. 38. 43. 42. 30. 37. 44. 28. 34. 35. 52. 32. 25. 31. 26. 33. 27.
 36. 40. 46. 41. 45. 19. 21. 24. nan 23. 22. 51. 48. 55. 50. 49. 20. 54.
 47. 56. 57. 63. 61. 62. 53. 58. 66. 59. 65. 74. 70.]


In [380]:
import pandas as pd

# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean and normalize the 'work percentage' feature in the original dataset
# Normalize the data by converting to lowercase and stripping whitespace
df['percentage_cleaned'] = df['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?']

# Step 2: Apply the percentage mapping to clean and standardize the 'work percentage' column
percentage_mapping = {
    'none': 0,
    '1-25%': 1,
    '26-50%': 2,
    '51-75%': 3,
    '76-100%': 4
}

# Apply the mapping
df['percentage_cleaned'] = df['percentage_cleaned'].replace(percentage_mapping)

# Step 3: Overwrite the original column with the cleaned 'percentage_cleaned'
df['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'] = df['percentage_cleaned']

# Step 4: Drop the temporary 'percentage_cleaned' column
df.drop(columns=['percentage_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame to a new CSV file
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Optionally, check if the changes are applied by printing unique values in the percentage column
print(df['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'].unique())

[nan  1.  4.  2.  3.]


In [381]:
import pandas as pd

# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature (no specific cleaning needed for this one)
df['employees_cleaned'] = df['How many employees does your company or organization have?']

# Step 2: Apply the mapping for 'How many employees does your company or organization have?'
employees_mapping = {
    '1-5': 0,
    '6-25': 1,
    '26-100': 2,
    '100-500': 3,
    '500-1000': 4,
    'More than 1000': 5
}

# Apply the mapping
df['employees_cleaned'] = df['employees_cleaned'].replace(employees_mapping)

# Step 3: Overwrite the original column with the cleaned 'employees_cleaned'
df['How many employees does your company or organization have?'] = df['employees_cleaned']

# Step 4: Drop the temporary 'employees_cleaned' column
df.drop(columns=['employees_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['How many employees does your company or organization have?'].unique())

[ 2.  1. nan  5.  3.  4.  0.]


In [382]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature (no specific cleaning needed for this one)
df['resources_cleaned'] = df['Do you know local or online resources to seek help for a mental health disorder?']

# Step 2: Apply the mapping
resources_mapping = {
    "No, I don't know any": 0,
    'I know some': 1,
    'Yes, I know several': 2
}

df['resources_cleaned'] = df['resources_cleaned'].replace(resources_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Do you know local or online resources to seek help for a mental health disorder?'] = df['resources_cleaned']

# Step 4: Drop the temporary 'resources_cleaned' column
df.drop(columns=['resources_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Do you know local or online resources to seek help for a mental health disorder?'].unique())

[nan  2.  1.  0.]


In [383]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature (no specific cleaning needed for this one)
df['willingness_cleaned'] = df['How willing would you be to share with friends and family that you have a mental illness?']

# Step 2: Apply the mapping
willingness_mapping = {
    'Not open at all': 0,
    'Somewhat not open': 1,
    'Neutral': 2,
    'Somewhat open': 3,
    'Very open': 4,
    'Not applicable to me (I do not have a mental illness)': -1
}

df['willingness_cleaned'] = df['willingness_cleaned'].replace(willingness_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['How willing would you be to share with friends and family that you have a mental illness?'] = df['willingness_cleaned']

# Step 4: Drop the temporary 'willingness_cleaned' column
df.drop(columns=['willingness_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['How willing would you be to share with friends and family that you have a mental illness?'].unique())

[ 3  2 -1  4  0  1]


In [384]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature
df['previous_resources_cleaned'] = df['Did your previous employers provide resources to learn more about mental health issues and how to seek help?']

# Step 2: Apply the mapping
previous_resources_mapping = {
    'None did': 0,
    'Some did': 1,
    'Yes, they all did': 2
}

df['previous_resources_cleaned'] = df['previous_resources_cleaned'].replace(previous_resources_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Did your previous employers provide resources to learn more about mental health issues and how to seek help?'] = df['previous_resources_cleaned']

# Step 4: Drop the temporary 'previous_resources_cleaned' column
df.drop(columns=['previous_resources_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Did your previous employers provide resources to learn more about mental health issues and how to seek help?'].unique())

[ 0.  1. nan  2.]


In [385]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature
df['negative_consequences_cleaned'] = df['Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?']

# Step 2: Apply the mapping
negative_consequences_mapping = {
    'None of them': 0,
    'Some of them': 1,
    'Yes, all of them': 2,
    "I don't know": 3
}

df['negative_consequences_cleaned'] = df['negative_consequences_cleaned'].replace(negative_consequences_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?'] = df['negative_consequences_cleaned']

# Step 4: Drop the temporary 'negative_consequences_cleaned' column
df.drop(columns=['negative_consequences_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?'].unique())

[ 0.  1. nan  2.]


In [386]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature
df['anonymity_cleaned'] = df['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?']

# Step 2: Apply the mapping
anonymity_protected_mapping = {
    'No': 0,
    'Sometimes': 1,
    'Yes, always': 2,
    "I don't know": 3
}

df['anonymity_cleaned'] = df['anonymity_cleaned'].replace(anonymity_protected_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?'] = df['anonymity_cleaned']

# Step 4: Drop the temporary 'anonymity_cleaned' column
df.drop(columns=['anonymity_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?'].unique())

[ 3.  2.  1.  0. nan]


In [387]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature
df['discussing_negative_consequences_cleaned'] = df['Do you think that discussing a mental health disorder with previous employers would have negative consequences?']

# Step 2: Apply the mapping
discussing_negative_consequences_mapping = {
    'None of them': 0,
    'Some of them': 1,
    'Yes, all of them': 2,
    "I don't know": 3
}

df['discussing_negative_consequences_cleaned'] = df['discussing_negative_consequences_cleaned'].replace(discussing_negative_consequences_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Do you think that discussing a mental health disorder with previous employers would have negative consequences?'] = df['discussing_negative_consequences_cleaned']

# Step 4: Drop the temporary 'discussing_negative_consequences_cleaned' column
df.drop(columns=['discussing_negative_consequences_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Do you think that discussing a mental health disorder with previous employers would have negative consequences?'].unique())

[ 1.  0.  3.  2. nan]


In [388]:
# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the feature
df['willing_to_discuss_cleaned'] = df['Would you have been willing to discuss a mental health issue with your direct supervisor(s)?']

# Step 2: Apply the mapping
willing_to_discuss_mapping = {
    'No, at none of my previous employers': 0,
    'Some of my previous employers': 1,
    'Yes, at all of my previous employers': 2,
    "I don't know": 3
}

df['willing_to_discuss_cleaned'] = df['willing_to_discuss_cleaned'].replace(willing_to_discuss_mapping)

# Step 3: Overwrite the original column with the cleaned version
df['Would you have been willing to discuss a mental health issue with your direct supervisor(s)?'] = df['willing_to_discuss_cleaned']

# Step 4: Drop the temporary 'willing_to_discuss_cleaned' column
df.drop(columns=['willing_to_discuss_cleaned'], inplace=True)

# Step 5: Save the updated DataFrame
df.to_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', index=False)

# Step 6: Check if the changes are applied by printing unique values in the column
print(df['Would you have been willing to discuss a mental health issue with your direct supervisor(s)?'].unique())

[ 1.  3.  0.  2. nan]


### Imputing Missing Values

In [389]:
import pandas as pd

# Load the dataset
df = pd.read_csv('01_mental-heath-in-tech-2016_20161114_standardized_features.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Separate columns by their data types
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Impute missing values for numerical columns using median
df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())

# Impute missing values for categorical columns using mode
for column in categorical_columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

# Optionally, save the cleaned dataset to a new CSV file
df.to_csv('02_mental-health-imputed_dataset.csv', index=False)

# Check if all missing values are imputed
missing_values_after_imputation = df.isnull().sum().sum()
print(f"Missing values after imputation: {missing_values_after_imputation}")

Missing values after imputation: 0


In [390]:
import pandas as pd

# Step 1: Load the updated dataset (the file where the cleaned gender feature was saved)
df_updated = pd.read_csv('02_mental-health-imputed_dataset.csv') #checking if the updated feature belongs to the cleaned dataset

# Step 2: Check the unique values in the 'What is your gender?' column
print("Unique values in the 'What is your gender?' column after cleaning:")
print(df_updated['What is your gender?'].unique())

#print(df_updated['Which of the following best describes your work position?'].unique())

Unique values in the 'What is your gender?' column after cleaning:
['Male' 'Female' 'Non-binary' 'Transgender Female' 'Other'
 'Prefer not to say' 'Transgender Male']


In [391]:
import pandas as pd

# Step 1: Load the updated dataset (the file where the cleaned gender feature was saved)
df_updated = pd.read_csv('02_mental-health-imputed_dataset.csv') #checking if the updated feature belongs to the cleaned dataset

# Step 2: Check the unique values in the 'What is your gender?' column
print("Unique values in the 'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?' column after cleaning:")
print(df_updated['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'].unique())

Unique values in the 'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?' column after cleaning:
[2. 1. 4. 3.]


### Feature Encoding

In [392]:
from sklearn.preprocessing import OneHotEncoder

In [393]:

# binary   'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?',
           # 'Have you been diagnosed with a mental health condition by a medical professional?',

#### Nominal Feature Encoding -> One-Hot Encoding

In [394]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer

In [395]:
# Load your dataset
df = pd.read_csv('02_mental-health-imputed_dataset.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

In [396]:
# Step 1: Extract categorical columns from the dataframe
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Define the nominal features as a subset of categorical columns
nominal_features = [
    'Does your employer provide mental health benefits as part of healthcare coverage?',
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
    'Do you feel that your employer takes mental health as seriously as physical health?',
    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?',
    'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?',
    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?',
    'If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?',
    'Do you believe your productivity is ever affected by a mental health issue?',
    'Have your previous employers provided mental health benefits?',
    'Were you aware of the options for mental health care provided by your previous employers?',
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
    'Did you feel that your previous employers took mental health as seriously as physical health?',
    'Would you be willing to bring up a physical health issue with a potential employer in an interview?',
    'Would you bring up a mental health issue with a potential employer in an interview?',
    'Do you feel that being identified as a person with a mental health issue would hurt your career?',
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?',
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?',
    'Have your observations of how another individual who discussed a mental health disorder made you less likely to reveal a mental health issue yourself in your current workplace?',
    'Do you have a family history of mental illness?',
    'Have you had a mental health disorder in the past?',
    'Do you currently have a mental health disorder?',
    'What is your gender?',
    'What country do you live in?',
    'What US state or territory do you live in?',
    'What country do you work in?',
    'What US state or territory do you work in?',
    'Do you know the options for mental health care available under your employer-provided coverage?',
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
    'Do you think that discussing a physical health issue with your employer would have negative consequences?',
    'Would you feel comfortable discussing a mental health disorder with your coworkers?'
]

#Initialize OneHotEncoder with handle_unknown set to 'ignore'
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the data
encoded_array = onehot.fit_transform(df[nominal_features])

# Get feature names
feature_names = onehot.get_feature_names_out(nominal_features)

# Create a new dataframe with encoded values
one_hot_df = pd.DataFrame(encoded_array, columns=feature_names)

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([df.reset_index(drop=True), one_hot_df.reset_index(drop=True)], axis=1)

## Drop the original nominal columns from the DataFrame
df_encoded = df_encoded.drop(nominal_features, axis=1)

df_encoded.to_csv('03_mental-health-encoding.csv', index=False)

one_hot_df

,Does your employer provide mental health benefits as part of healthcare coverage?_I don't know,Does your employer provide mental health benefits as part of healthcare coverage?_No,Does your employer provide mental health benefits as part of healthcare coverage?_Not eligible for coverage / N/A,Does your employer provide mental health benefits as part of healthcare coverage?_Yes,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?_Maybe,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?_No,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?_Yes,Do you feel that your employer takes mental health as seriously as physical health?_I don't know,Do you feel that your employer takes mental health as seriously as physical health?_No,Do you feel that your employer takes mental health as seriously as physical health?_Yes,...,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?_Yes,Do you think that discussing a mental health disorder with your employer would have negative consequences?_Maybe,Do you think that discussing a mental health disorder with your employer would have negative consequences?_No,Do you think that discussing a mental health disorder with your employer would have negative consequences?_Yes,Do you think that discussing a physical health issue with your employer would have negative consequences?_Maybe,Do you think that discussing a physical health issue with your employer would have negative consequences?_No,Do you think that discussing a physical health issue with your employer would have negative consequences?_Yes,Would you feel comfortable discussing a mental health disorder with your coworkers?_Maybe,Would you feel comfortable discussing a mental health disorder with your coworkers?_No,Would you feel comfortable discussing a mental health disorder with your coworkers?_Yes
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1429,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1430,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1431,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [397]:
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Binary encoding for 'Have you heard of or observed negative consequences...'
df['Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'] = df['Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'].map({'No': 0, 'Yes': 1})

# Binary encoding for 'Have you been diagnosed with a mental health condition by a medical professional?'
df['Have you been diagnosed with a mental health condition by a medical professional?'] = df['Have you been diagnosed with a mental health condition by a medical professional?'].map({'No': 0, 'Yes': 1})

# Save the updated DataFrame to a new CSV file
df.to_csv('03_mental-health-encoding.csv', index=False)

# Check the encoded columns
print(df[['Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?', 
         'Have you been diagnosed with a mental health condition by a medical professional?']].head())

   Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?  \
0                                                  0                                                                                     
1                                                  0                                                                                     
2                                                  0                                                                                     
3                                                  0                                                                                     
4                                                  0                                                                                     

   Have you been diagnosed with a mental health condition by a medical professional?  
0                                                  1                                

In [398]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Define multi-label features
multi_label_features = [
    'If yes, what condition(s) have you been diagnosed with?',
    'If maybe, what condition(s) do you believe you have?',
    'If so, what condition(s) were you diagnosed with?',
    'Which of the following best describes your work position?'
]

# Apply MultiLabelBinarizer to each feature
mlb = MultiLabelBinarizer()

for feature in multi_label_features:
    # Split the conditions based on the '|' delimiter
    df[feature] = df[feature].fillna('').str.split('|')
    
    # Apply MultiLabelBinarizer and create new columns for each possible condition
    mlb_result = pd.DataFrame(mlb.fit_transform(df[feature]), columns=mlb.classes_)
    
    # Concatenate the original dataframe with the new multi-label binary columns
    df = pd.concat([df, mlb_result], axis=1)

# Drop the original multi-label columns if desired
df.drop(columns=multi_label_features, inplace=True)

# Save the transformed dataset
df.to_csv('03_mental-health-encoding.csv', index=False)


print(mlb_result.head())

   Back-end Developer  Designer  Dev Evangelist/Advocate  DevOps/SysAdmin  \
0                   1         0                        0                0   
1                   1         0                        0                0   
2                   1         0                        0                0   
3                   0         0                        0                0   
4                   1         0                        1                1   

   Executive Leadership  Front-end Developer  HR  One-person shop  Other  \
0                     0                    0   0                0      0   
1                     0                    1   0                0      0   
2                     0                    0   0                0      0   
3                     0                    0   0                0      0   
4                     1                    1   0                0      0   

   Sales  Supervisor/Team Lead  Support  
0      0                     0        

#### Ordinal Feature Encoding -> Ordinal Encoding Technique

In [399]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# List of ordinal feature columns (replace these with the actual column names from your dataset)
ordinal_features = [
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
    'If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
    'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?',
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
    'Would you have been willing to discuss a mental health issue with your previous co-workers?',
    'Do you work remotely?'
]

# Step 1: Check if all ordinal features are present
missing_features = [col for col in ordinal_features if col not in df.columns]
if missing_features:
    print(f"The following ordinal features are missing from the dataset: {missing_features}")
else:
    # Step 2: Initialize OrdinalEncoder
    ordinal_encoder = OrdinalEncoder()

    # Step 3: Apply Ordinal Encoding to the ordinal features
    df[ordinal_features] = ordinal_encoder.fit_transform(df[ordinal_features])

    # Step 4: Save the updated dataset back to the same CSV file
    df.to_csv('03_mental-health-encoding.csv', index=False)

    print("Ordinal features encoded successfully and saved to '03_mental-health-encoding.csv'.")
    print(df[ordinal_features].head())

Ordinal features encoded successfully and saved to '03_mental-health-encoding.csv'.
   If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:  \
0                                                5.0                                                             
1                                                3.0                                                             
2                                                1.0                                                             
3                                                3.0                                                             
4                                                1.0                                                             

   If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?  \
0                                                1.0                                                            

#### Textual Features Encoding --->  Technique

In [274]:
%pip install spacy
!python -m spacy download en_core_web_sm

  Using cached numpy-2.0.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp312-cp312-macosx_14_0_arm64.whl (5.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.
tensorflow 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.9 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [275]:
pip install gensim

  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [400]:
import spacy
import re
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Load spaCy English model
nlp = spacy.load('en_core_web_sm')

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Step 1: Clean the text data by removing numbers (for 'Why or why not?' and 'Why or why not?.1')
df['Why or why not?_cleaned'] = df['Why or why not?'].fillna('').apply(lambda x: re.sub(r'\d+', '', x))
df['Why or why not?.1_cleaned'] = df['Why or why not?.1'].fillna('').apply(lambda x: re.sub(r'\d+', '', x))

# Step 2: Tokenize the text using spaCy
df['Why_or_why_not_tokenized'] = df['Why or why not?_cleaned'].apply(lambda x: [token.text.lower() for token in nlp(x)])
df['Why_or_why_not_1_tokenized'] = df['Why or why not?.1_cleaned'].apply(lambda x: [token.text.lower() for token in nlp(x)])

# Step 3: Prepare the data for Doc2Vec
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['Why_or_why_not_tokenized'])]
documents_1 = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['Why_or_why_not_1_tokenized'])]

# Step 4: Train the Doc2Vec model
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4)
model_1 = Doc2Vec(documents_1, vector_size=100, window=5, min_count=1, workers=4)

# Step 5: Convert the Doc2Vec vectors to DataFrame
why_doc2vec = [model.infer_vector(doc.words) for doc in documents]
why_1_doc2vec = [model_1.infer_vector(doc.words) for doc in documents_1]

why_doc2vec_df = pd.DataFrame(why_doc2vec, columns=['why_vec_' + str(i) for i in range(100)])
why_1_doc2vec_df = pd.DataFrame(why_1_doc2vec, columns=['why_1_vec_' + str(i) for i in range(100)])

# Concatenate the new DataFrame with your original dataset
df = pd.concat([df, why_doc2vec_df, why_1_doc2vec_df], axis=1)

# Step 6: Drop the original text columns and temporary cleaned columns
df.drop(columns=['Why or why not?', 'Why or why not?.1', 'Why or why not?_cleaned', 'Why or why not?.1_cleaned', 'Why_or_why_not_tokenized', 'Why_or_why_not_1_tokenized'], inplace=True)

# Step 7: Save the updated DataFrame to a new CSV file
df.to_csv('03_mental-health-encoding.csv', index=False)

# Check if the changes are applied by printing the first few rows
print(df.head())

   Are you self-employed?  \
0                       0   
1                       0   
2                       0   
3                       1   
4                       0   

   How many employees does your company or organization have?  \
0                                                2.0            
1                                                1.0            
2                                                1.0            
3                                                3.0            
4                                                1.0            

   Is your employer primarily a tech company/organization?  \
0                                                1.0         
1                                                1.0         
2                                                1.0         
3                                                1.0         
4                                                0.0         

   Is your primary role within your company related to tech/I

#### Check-Ups

In [240]:
import pandas as pd

# Load the dataset after encoding
df = pd.read_csv('03_mental-health-encoding.csv')

# Identify columns that are still categorical (object type)
categorical_cols = df.select_dtypes(include=['object']).columns

# For each categorical column, print the unique values
for col in categorical_cols:
    print(f"Feature: {col}")
    print(f"Unique categories: {df[col].unique()}")
    print("\n")

In [241]:
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Filter the features with missing values
features_with_missing_values = missing_values[missing_values > 0]

# Print features with missing values
print(features_with_missing_values)

Series([], dtype: int64)


In [242]:
import pandas as pd

df = pd.read_csv('03_mental-health-encoding.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')
# Get the number of columns
num_columns = df.shape[1]

print(f"The dataset has {num_columns} columns.")

The dataset has 628 columns.


In [243]:
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv')

# List non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['float64', 'int64']).columns

# Print the non-numeric columns
print("Non-numeric columns in the dataset:")
print(non_numeric_columns)

Non-numeric columns in the dataset:
Index([], dtype='object')


#### Nominal Feature Encoding: trying out different approaches to seek the best one

In [220]:
%pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [70]:
#THE BEST OUTPUT SO FAR!!!!!
onehot = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the data
encoded_array = onehot.fit_transform(df[nominal_features])

# Get feature names
feature_names = onehot.get_feature_names_out(nominal_features)

# Create a new dataframe with encoded values
encoded_df = pd.DataFrame(encoded_array, columns=feature_names)

# Simplify column names
encoded_df.columns = [f"{col.split('_')[0]}_{col.split('_')[-1]}" for col in encoded_df.columns]

encoded_df

,How many employees does your company or organization have?_0,How many employees does your company or organization have?_1,How many employees does your company or organization have?_2,How many employees does your company or organization have?_3,How many employees does your company or organization have?_4,How many employees does your company or organization have?_5,Does your employer provide mental health benefits as part of healthcare coverage?_0,Does your employer provide mental health benefits as part of healthcare coverage?_1,Does your employer provide mental health benefits as part of healthcare coverage?_2,Does your employer provide mental health benefits as part of healthcare coverage?_3,...,Which of the following best describes your work position?_254,Which of the following best describes your work position?_255,Which of the following best describes your work position?_256,Which of the following best describes your work position?_257,Which of the following best describes your work position?_258,Which of the following best describes your work position?_259,Which of the following best describes your work position?_260,Which of the following best describes your work position?_261,Which of the following best describes your work position?_262,Which of the following best describes your work position?_263
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1429,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1430,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1431,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Checking Categories of List of Features 

In [102]:
# features with many categories
'If yes, what condition(s) have you been diagnosed with?'
'If maybe, what condition(s) do you believe you have?'
'If so, what condition(s) were you diagnosed with?'
'Which of the following best describes your work position?'

In [ ]:
#ordinal


In [ ]:

#binary
'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'
'Have you been diagnosed with a mental health condition by a medical professional?'

In [158]:
import pandas as pd

# Load the dataset
df = pd.read_csv('02_mental-health-imputed_dataset.csv', delimiter=",", header=0, on_bad_lines='skip', encoding='utf-8')

# Display unique values in the 'Why or why not?' column
print("Unique categories in 'Why or why not?':")
print(df['Why or why not?'].unique())

# Display unique values in the 'Why or why not?.1' column
print("\nUnique categories in 'Why or why not?.1':")
print(df['Why or why not?.1'].unique())

Unique categories in 'Why or why not?':
['Fear of discrimination'
 'It would depend on the health issue. If there is a health issue that would not immediately affect my job performance, such as diabetes, I would not bring it up during the interview. If it was something more severe, such as narcolepsy, I might bring it up depending on how controlled it was.'
 'They would provable need to know, to Judge if I can do my job or not. '
 ... ' QF'
 'Fear that doing so would cause the employer to factor in additional health insurance expense.'
 'Stigma with some diseases']

Unique categories in 'Why or why not?.1':
['Stigma'
 'While mental health has become a more prominent issue recently, I feel like there is still a lot of stigma surrounding it. At this point, with our culture, I would not bring it up. I hope that within the next 20-30 years that stigma will be gone and it can be brought up safely in an interview.'
 'Stigma, mainly. ' ... 'E'
 'Fear that the employer would consider additiona

In [401]:
# Check the data type of 'What is your age?' column
df = pd.read_csv('03_mental-health-encoding.csv')

print(df['What is your age?'].dtype)

float64


In [402]:
import pandas as pd

# Load the dataset
df = pd.read_csv('03_mental-health-encoding.csv')

# Check for missing values in the dataset
missing_values = df.isnull().sum()

# Display columns with missing values
print(missing_values[missing_values > 0])

Series([], dtype: int64)
